# Lottery Dataset

First, we import the packages we need

In [1]:
import pandas as pd

Second, we import the actual dataset. The dataset comes from a txt-file which has uneven spaces between columns and a comment at the beginning. We use an argumet 'skiprows = 4' to remove the comment. 

In [100]:
data = pd.read_csv('LotteryData.txt', sep = " ", skiprows = 4, header = None, index_col = 0)

In [101]:
print(data.shape)

(514, 73)


We see that the dimensions of the dataset has correct number of rows (514), however the number of columns is incorrect due to uneven number of spaces in the original file. We look at the data and see that some columns have NaN values: 

In [102]:
print(data.head())

     1   2   3   4     5    6    7     8   9   10 ...  64  65  66  67  68  69  \
0                                                 ...                           
514 NaN NaN NaN NaN  Sat.  Mar  10,  2018 NaN NaN ... NaN NaN NaN NaN  22 NaN   
513 NaN NaN NaN NaN  Wed.  Mar  07,  2018 NaN NaN ... NaN NaN NaN NaN  18 NaN   
512 NaN NaN NaN NaN  Sat.  Mar  03,  2018 NaN NaN ... NaN NaN NaN NaN   5 NaN   
511 NaN NaN NaN NaN  Wed.  Feb  28,  2018 NaN NaN ... NaN NaN NaN NaN  16 NaN   
510 NaN NaN NaN NaN  Sat.  Feb  24,  2018 NaN NaN ... NaN NaN NaN NaN   6 NaN   

     70  71  72  73  
0                    
514 NaN NaN NaN NaN  
513 NaN NaN NaN NaN  
512 NaN NaN NaN NaN  
511 NaN NaN NaN NaN  
510 NaN NaN NaN NaN  

[5 rows x 73 columns]


We remove columns with NaNs using .dropna() method. 

In [103]:
data = data.dropna(axis = 1, how = 'all')
print(data.head())
print(data.shape)

       5    6    7     8   18  28  38  48  58  68
0                                                
514  Sat.  Mar  10,  2018  43  44  54  61  69  22
513  Wed.  Mar  07,  2018   6  13  19  36  51  18
512  Sat.  Mar  03,  2018  13  17  25  36  40   5
511  Wed.  Feb  28,  2018  12  30  59  65  69  16
510  Sat.  Feb  24,  2018  24  25  38  62  63   6
(514, 10)


In [104]:
data.columns = ['WeekDay', 'Month', 'Day', 'Year', '1', '2', '3', '4', '5', 'PB']
data.index.name = 'Draw'

In [105]:
print(data.head())

     WeekDay Month  Day  Year   1   2   3   4   5  PB
Draw                                                 
514     Sat.   Mar  10,  2018  43  44  54  61  69  22
513     Wed.   Mar  07,  2018   6  13  19  36  51  18
512     Sat.   Mar  03,  2018  13  17  25  36  40   5
511     Wed.   Feb  28,  2018  12  30  59  65  69  16
510     Sat.   Feb  24,  2018  24  25  38  62  63   6


In [118]:
data['Day'] = data['Day'].apply(lambda x: str(x).rstrip(','))
print(data.head())

     WeekDay Month Day  Year   1   2   3   4   5  PB
Draw                                                
514     Sat.   Mar  10  2018  43  44  54  61  69  22
513     Wed.   Mar  07  2018   6  13  19  36  51  18
512     Sat.   Mar  03  2018  13  17  25  36  40   5
511     Wed.   Feb  28  2018  12  30  59  65  69  16
510     Sat.   Feb  24  2018  24  25  38  62  63   6


In [124]:
from time import strptime

data['Month'] = data['Month'].apply(lambda x: strptime(x,'%b').tm_mon)


In [130]:
data['Date'] = pd.to_datetime(data[['Year', 'Month', 'Day']], format = '%Y%M%D')
data = data.drop(['Year', 'Month', 'Day'], axis =1)
print(data.info())

KeyError: "['Year' 'Month' 'Day'] not in index"

In [133]:
data = data[['Date', '1', '2', '3', '4', '5', 'PB']]
print(data.head())

           Date   1   2   3   4   5  PB
Draw                                   
514  2018-03-10  43  44  54  61  69  22
513  2018-03-07   6  13  19  36  51  18
512  2018-03-03  13  17  25  36  40   5
511  2018-02-28  12  30  59  65  69  16
510  2018-02-24  24  25  38  62  63   6


The data is clean